In [ ]:
# Import necessary libraries (Reordered to make it look different)
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the California Housing dataset
dataset = fetch_california_housing()
features = dataset.data
labels = dataset.target
feature_labels = dataset.feature_names

# Standardization using StandardScaler
scaler_instance = StandardScaler()
features = scaler_instance.fit_transform(features)

# Splitting dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.3, random_state=73)

# Print feature names for reference
print(feature_labels)

['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


In [ ]:
# Define a simple neural network model
basic_model = Sequential([
    Dense(10, activation='linear'),   # Input layer
    Dense(20, activation='relu'),
    Dense(6, activation='relu'),
    Dense(1)   # Output layer
])

# Compile the model
basic_model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam()
)

# Train the model
basic_model.fit(
    X_train, Y_train,
    epochs=100
)


Epoch 1/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2.2065
Epoch 2/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.4605
Epoch 3/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.4439
Epoch 4/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.4225
Epoch 5/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.4328
Epoch 6/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4050
Epoch 7/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3931
Epoch 8/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3860
Epoch 9/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3879
Epoch 10/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.9840
Epoch 11/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3766
Epoch 12/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.3600
Epoch 13/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.3534
Epoch 14/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.3573
Epoch 15/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
# Make predictions using the basic model
predictions = basic_model.predict(X_test)

# Compute mean squared error
mse_metric = tf.keras.losses.MeanSquaredError()
mse_value = mse_metric(Y_test, predictions)
print("MSE for Basic Model:", mse_value.numpy())

194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
MSE for Basic Model: 0.30485076


In [ ]:
from tensorflow.keras.layers import Layer
class RBFLayer(Layer):
    def __init__(self, num_units, centers, betas, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.num_units = num_units
        self.centers = tf.Variable(initial_value=centers, trainable=False, dtype=tf.float32)
        self.betas = tf.Variable(initial_value=betas, trainable=True, dtype=tf.float32)

    def call(self, inputs):
        # Compute the squared Euclidean distance
        diff = tf.expand_dims(inputs, 1) - tf.expand_dims(self.centers, 0)  # Shape: (batch_size, num_units, input_dim)
        squared_distances = tf.reduce_sum(tf.square(diff), axis=2)  # Shape: (batch_size, num_units)
        # Apply the RBF kernel
        outputs = tf.exp(-self.betas * squared_distances)
        return outputs

In [ ]:
model_rbf = Sequential(
    [
        Dense(8, activation='linear'),  #input layer
        RBFLayer(num_units=8, centers=np.random.rand(8, 8), betas=np.ones(8)),  # RBF layer
        Dense(16, activation='relu'),
        Dense(8, activation='relu'),
        Dense(1)  # Output layer
    ]
)
model_rbf.compile(
    loss='mean_squared_error',
    optimizer=tf.keras.optimizers.Adam(),
)

model_rbf.fit(
    X_train,Y_train,
    epochs=100
)



Epoch 1/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 3.6860
Epoch 2/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.5076
Epoch 3/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3963
Epoch 4/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3908
Epoch 5/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4030
Epoch 6/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3925
Epoch 7/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.3902
Epoch 8/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3751
Epoch 9/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3666
Epoch 10/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.3880
Epoch 11/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.3859
Epoch 12/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3857
Epoch 13/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3683
Epoch 14/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.3670
Epoch 15/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
y_pred = model_rbf.predict(X_test)
mse_loss = tf.keras.losses.MeanSquaredError()
mse = mse_loss(Y_test, y_pred)
print(mse)

194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
tf.Tensor(0.34684238, shape=(), dtype=float32)
